# Exploratory Data Analysis for Urea Pricing
---

## Feature Engineering + EDA
---

In [ ]:
import os
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from pathlib import Path

print("All necessary libraries imported successfully.")

## Configs and Paths:
---

In [ ]:
DATA_DIR = os.path.abspath("./data")
DATA_DIR = Path(DATA_DIR)
RAW_DIR = DATA_DIR / "raw"

paths = {
    "merged": DATA_DIR / "merged_data.csv",
    "pink": RAW_DIR / "pink_sheet_monthly.csv",
    "ptax": RAW_DIR / "bcb_ptax_usdbrl_1990-01_2025-12.csv",
    "gscpi": RAW_DIR / "nyfed_gscpi.csv",
    "oni": RAW_DIR / "noaa_oni.csv",
    "gpr": RAW_DIR / "gpr.csv",
    "events": RAW_DIR / "main_events.csv",
    "trade": DATA_DIR / "urea_trade_features.csv",
    "india": DATA_DIR / "india_urea_hs6_by_partner_wits.csv",
}

OUT_DIR = DATA_DIR / "eda_outputs"
FIG_DIR = OUT_DIR / "figures"
TAB_DIR = OUT_DIR / "tables"
FIG_DIR.mkdir(parents=True, exist_ok=True)
TAB_DIR.mkdir(parents=True, exist_ok=True)

UREA_COL = "urea_usd"

In [ ]:
def to_month_start(dt_series: pd.Series) -> pd.Series:
    s = pd.to_datetime(dt_series, errors="coerce")
    return s.dt.to_period("M").dt.to_timestamp()

def savefig(fig, filename: str, dpi: int = 160):
    fig.tight_layout()
    fig.savefig(FIG_DIR / filename, dpi=dpi)
    plt.close(fig)

def add_seasonality(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    m = out.index.month.astype(int)
    out["month"] = m
    out["month_sin"] = np.sin(2 * np.pi * m / 12)
    out["month_cos"] = np.cos(2 * np.pi * m / 12)
    return out


def add_lags(df: pd.DataFrame, cols: List[str], lags: List[int]) -> pd.DataFrame:
    out = df.copy()
    new_cols = {}
    for c in cols:
        if c not in out.columns:
            continue
        for L in lags:
            new_cols[f"{c}_lag{L}"] = out[c].shift(L)
    if new_cols:
        out = pd.concat([out, pd.DataFrame(new_cols)], axis=1)
    return out


def add_rolling(df: pd.DataFrame, cols: List[str], windows: List[int]) -> pd.DataFrame:
    out = df.copy()
    new_cols = {}
    for c in cols:
        if c not in out.columns:
            continue
        for w in windows:
            new_cols[f"{c}_ma{w}"] = out[c].rolling(w).mean()
    if new_cols:
        out = pd.concat([out, pd.DataFrame(new_cols)], axis=1)
    return out


def lag_correlation_table(df: pd.DataFrame, target: str, features: List[str], lags: List[int]) -> pd.DataFrame:
    rows = []
    y = df[target]
    for f in features:
        if f not in df.columns:
            continue
        for L in lags:
            corr = y.corr(df[f].shift(L))
            rows.append({"feature": f, "lag": L, "corr": corr})
    out = pd.DataFrame(rows).dropna()
    out["abs_corr"] = out["corr"].abs()
    return out.sort_values(["abs_corr"], ascending=False).reset_index(drop=True)


def train_feature_importance_ts(
    df: pd.DataFrame,
    target: str,
    drop_cols: Optional[List[str]] = None,
    n_splits: int = 5,
    random_state: int = 42,
) -> Tuple[pd.DataFrame, Dict[str, float]]:
    drop_cols = drop_cols or []
    data = df.dropna(subset=[target]).copy()
    X = data.drop(columns=[target] + drop_cols, errors="ignore")
    y = data[target].copy()

    # Remover colunas não-numéricas
    X = X.select_dtypes(include=[np.number]).copy()

    # Tirar linhas com NA após lags/rolling
    mask = X.notna().all(axis=1) & y.notna()
    X = X[mask]
    y = y[mask]

    tscv = TimeSeriesSplit(n_splits=n_splits)
    fold_metrics = {"mae": [], "rmse": [], "r2": []}

    # Modelo simples e interpretável via importâncias + permutação
    model = RandomForestRegressor(
        n_estimators=600,
        max_depth=None,
        min_samples_leaf=2,
        random_state=random_state,
        n_jobs=-1,
    )

    # Avaliação em folds e treino final no full (para importâncias)
    for tr, te in tscv.split(X):
        Xtr, Xte = X.iloc[tr], X.iloc[te]
        ytr, yte = y.iloc[tr], y.iloc[te]
        model.fit(Xtr, ytr)
        pred = model.predict(Xte)
        fold_metrics["mae"].append(mean_absolute_error(yte, pred))
        fold_metrics["rmse"].append(np.sqrt(mean_squared_error(yte, pred)))
        fold_metrics["r2"].append(r2_score(yte, pred))

    metrics_summary = {
        "mae_mean": float(np.mean(fold_metrics["mae"])),
        "rmse_mean": float(np.mean(fold_metrics["rmse"])),
        "r2_mean": float(np.mean(fold_metrics["r2"])),
        "n_obs": int(len(X)),
        "n_features": int(X.shape[1]),
    }

    # Treino final para importâncias
    model.fit(X, y)

    imp_rf = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)

    perm = permutation_importance(model, X, y, n_repeats=15, random_state=random_state, n_jobs=-1)
    imp_perm = pd.Series(perm.importances_mean, index=X.columns).sort_values(ascending=False)

    out = pd.DataFrame({
        "feature": X.columns,
        "rf_importance": imp_rf.reindex(X.columns).values,
        "perm_importance": imp_perm.reindex(X.columns).values,
    }).sort_values(["perm_importance", "rf_importance"], ascending=False)

    return out.reset_index(drop=True), metrics_summary


def plot_series(df: pd.DataFrame, cols: List[str], outpath: str, title: str) -> None:
    plt.figure(figsize=(12, 5))
    for c in cols:
        if c in df.columns:
            plt.plot(df.index, df[c], label=c)
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outpath, dpi=140)
    plt.close()


def plot_bar(df: pd.DataFrame, x: str, y: str, outpath: str, title: str, top_n: int = 20) -> None:
    d = df.head(top_n).copy()
    plt.figure(figsize=(10, 6))
    plt.barh(d[x][::-1], d[y][::-1])
    plt.title(title)
    plt.tight_layout()
    plt.savefig(outpath, dpi=140)
    plt.close()

def dataset_overview(name: str, df: pd.DataFrame, date_col: str = None):
    out = {
        "dataset": name,
        "rows": len(df),
        "cols": df.shape[1],
    }
    if date_col and date_col in df.columns:
        d = pd.to_datetime(df[date_col], errors="coerce")
        out["min_date"] = d.min()
        out["max_date"] = d.max()
    else:
        out["min_date"] = pd.NaT
        out["max_date"] = pd.NaT
    out["missing_cells_pct"] = round(float(df.isna().mean().mean() * 100), 3)
    return out


def zscore(s: pd.Series) -> pd.Series:
    s = s.astype(float)
    return (s - s.mean()) / s.std(ddof=0)

## Loading all datasets:
---

In [ ]:
df_merged = pd.read_csv(paths["merged"])
df_merged["date"] = pd.to_datetime(df_merged["date"], errors="coerce")
df_merged = df_merged.sort_values("date").reset_index(drop=True)

df_pink = pd.read_csv(paths["pink"])
df_pink["date"] = pd.to_datetime(df_pink["date"], errors="coerce")

df_ptax = pd.read_csv(paths["ptax"])
df_ptax["date"] = pd.to_datetime(df_ptax["date"], errors="coerce")

df_gscpi = pd.read_csv(paths["gscpi"])
df_gscpi["date"] = pd.to_datetime(df_gscpi["date"], errors="coerce")

df_oni = pd.read_csv(paths["oni"])
df_oni["date"] = pd.to_datetime(df_oni["date"], errors="coerce")

df_gpr = pd.read_csv(paths["gpr"])
df_gpr["date"] = pd.to_datetime(df_gpr["date"], errors="coerce")

df_events = pd.read_csv(paths["events"])
df_events["date"] = pd.to_datetime(df_events["date"], errors="coerce")

df_trade = pd.read_csv(paths["trade"])
df_trade["date"] = pd.to_datetime(df_trade["date"], errors="coerce")
mask = df_trade["flowDesc"] == "Re-import"
df_trade = df_trade[~mask].copy()

df_trade_pivoted = df_trade.pivot_table(
    index='date',
    columns='flowDesc',
    values=['CHN_tonnes', 'CHN_value_usd', 'BRA_tonnes', 'BRA_value_usd'],
    aggfunc='first'
)
df_trade_pivoted.columns = ['_'.join(col).strip() for col in df_trade_pivoted.columns.values]

df_india = pd.read_csv(paths["india"])
df_india["date"] = pd.to_datetime(df_india["year"].astype(str) + "-01-01", errors="coerce")

df_merged = df_merged.set_index("date")
df_merged = df_merged.drop(columns=["index"])

if UREA_COL not in df_merged.columns:
    raise ValueError(f"Coluna-alvo não encontrada: {UREA_COL}")

# Adiciona features de comércio internacional ao dataset principal
df_merged = df_merged.merge(df_trade_pivoted, left_index=True, right_index=True, how="left")
df_merged['urea_brl'] = df_merged[UREA_COL] * df_merged['usdbrl']

df_merged.to_csv(DATA_DIR / "merged_data_with_trade.csv")


# Aplica filtro de data inicial como 1995-01-01.
df_merged = df_merged[df_merged.index >= pd.to_datetime("1995-01-01")]


In [ ]:
overview = pd.DataFrame([
    dataset_overview("merged_data.csv", df_merged.reset_index(), "date"),
    dataset_overview("pink_sheet_monthly.csv", df_pink, "date"),
    dataset_overview("ptax_usdbrl.csv", df_ptax, "date"),
    dataset_overview("nyfed_gscpi.csv", df_gscpi, "date"),
    dataset_overview("noaa_oni.csv", df_oni, "date"),
    dataset_overview("gpr.csv", df_gpr, "date"),
    dataset_overview("main_events.csv", df_events, "date"),
    dataset_overview("urea_trade_features.csv", df_trade, "date"),
    dataset_overview("india_urea_hs6_by_partner_wits.csv", df_india, "date"),
])

overview.to_csv(TAB_DIR / "00_dataset_overview.csv", index=False)

In [ ]:
from scipy.stats import entropy

# Selecionar as features de interesse
features = ["urea_usd", "crude_oil_usd", "natural_gas_usd", "usdbrl", "gscpi", "gpr"]
df_features = df_merged[features].dropna()

# Calcular a entropia para cada feature
entropy_results = {}

for col in features:
    # Normalizar os dados para criar uma distribuição de probabilidade
    # Usando histograma para discretizar valores contínuos
    hist, bin_edges = np.histogram(df_features[col], bins=30, density=True)
    
    # Normalizar para soma = 1 (distribuição de probabilidade)
    hist = hist / hist.sum()
    
    # Remover bins vazios para evitar log(0)
    hist = hist[hist > 0]
    
    # Calcular entropia
    ent = entropy(hist, base=2)  # base=2 para bits, base=e para nats
    entropy_results[col] = ent

# Criar DataFrame com resultados
entropy_df = pd.DataFrame.from_dict(
    entropy_results, 
    orient='index', 
    columns=['entropy']
).sort_values('entropy', ascending=False)

print("=" * 60)
print("ENTROPIA DAS FEATURES (em bits)")
print("=" * 60)
print(entropy_df)
print("\nInterpretação:")
print("- Maior entropia = maior incerteza/variabilidade")
print("- Menor entropia = mais previsível/concentrada")

# Visualização
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de barras da entropia
ax1.barh(entropy_df.index, entropy_df['entropy'], color='steelblue')
ax1.set_xlabel('Entropia (bits)', fontsize=11)
ax1.set_title('Entropia por Feature\n(Medida de incerteza/variabilidade)', fontsize=12, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Distribuições normalizadas para comparação visual
for i, col in enumerate(features):
    hist, bin_edges = np.histogram(df_features[col], bins=30, density=True)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    ax2.plot(bin_centers, hist / hist.sum(), label=f"{col} (H={entropy_results[col]:.2f})", alpha=0.7)

ax2.set_xlabel('Valor normalizado', fontsize=10)
ax2.set_ylabel('Densidade de probabilidade', fontsize=10)
ax2.set_title('Distribuições de Probabilidade das Features', fontsize=12, fontweight='bold')
ax2.legend(fontsize=8, loc='best')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Salvar resultados
# entropy_df.to_csv(TAB_DIR / "entropy_analysis.csv")
# print(f"\nResultados salvos em: {TAB_DIR / 'entropy_analysis.csv'}")

In [ ]:
from scipy.stats import entropy

# Selecionar as features de interesse
features = ["urea_usd", "crude_oil_usd", "natural_gas_usd", "usdbrl", "gscpi", "gpr"]
df_features = df_merged[features].dropna()

# Calcular o Índice de Shannon (Entropia de Shannon) para cada feature
shannon_results = {}

for col in features:
    # Normalizar os dados para criar uma distribuição de probabilidade
    # Usando histograma para discretizar valores contínuos
    hist, bin_edges = np.histogram(df_features[col], bins=30, density=True)
    
    # Normalizar para soma = 1 (distribuição de probabilidade)
    hist = hist / hist.sum()
    
    # Remover bins vazios para evitar log(0)
    hist = hist[hist > 0]
    
    # Calcular Índice de Shannon (entropia em nats - base natural)
    shannon_index = entropy(hist, base=np.e)  # base=e para nats (índice de Shannon clássico)
    shannon_results[col] = shannon_index

# Criar DataFrame com resultados
shannon_df = pd.DataFrame.from_dict(
    shannon_results, 
    orient='index', 
    columns=['shannon_index']
).sort_values('shannon_index', ascending=False)

print("=" * 70)
print("ÍNDICE DE SHANNON (ENTROPIA DE SHANNON) - em nats")
print("=" * 70)
print(shannon_df)
print("\nInterpretação do Índice de Shannon:")
print("- Maior valor = maior diversidade/incerteza na distribuição")
print("- Menor valor = maior concentração/previsibilidade")
print("- Unidade: nats (logaritmo natural)")
print("- Conversão: 1 nat ≈ 1.443 bits")

# Adicionar versão em bits para comparação
shannon_df['shannon_bits'] = shannon_df['shannon_index'] / np.log(2)

print("\n" + "=" * 70)
print("ÍNDICE DE SHANNON - Comparação (nats vs bits)")
print("=" * 70)
print(shannon_df)

# Visualização
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# 1. Gráfico de barras horizontal - Índice de Shannon (nats)
ax1.barh(shannon_df.index, shannon_df['shannon_index'], color='steelblue', alpha=0.8)
ax1.set_xlabel('Índice de Shannon (nats)', fontsize=11)
ax1.set_title('Índice de Shannon por Feature\n(Medida de diversidade/entropia)', 
              fontsize=12, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# 2. Gráfico de barras horizontal - Índice de Shannon (bits)
ax2.barh(shannon_df.index, shannon_df['shannon_bits'], color='coral', alpha=0.8)
ax2.set_xlabel('Índice de Shannon (bits)', fontsize=11)
ax2.set_title('Índice de Shannon por Feature\n(Convertido para bits)', 
              fontsize=12, fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

# 3. Distribuições de probabilidade (densidade)
for i, col in enumerate(features):
    hist, bin_edges = np.histogram(df_features[col], bins=30, density=True)
    hist_prob = hist / hist.sum()
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    ax3.plot(bin_centers, hist_prob, 
             label=f"{col}\n(H={shannon_results[col]:.3f} nats)", 
             alpha=0.7, linewidth=2)

ax3.set_xlabel('Valor (normalizado)', fontsize=10)
ax3.set_ylabel('Densidade de probabilidade', fontsize=10)
ax3.set_title('Distribuições de Probabilidade das Features\n(base para cálculo do Índice de Shannon)', 
              fontsize=12, fontweight='bold')
ax3.legend(fontsize=8, loc='best')
ax3.grid(alpha=0.3)

# 4. Comparação visual: nats vs bits
x_pos = np.arange(len(shannon_df))
width = 0.35

ax4.bar(x_pos - width/2, shannon_df['shannon_index'], width, 
        label='Shannon (nats)', color='steelblue', alpha=0.8)
ax4.bar(x_pos + width/2, shannon_df['shannon_bits'], width, 
        label='Shannon (bits)', color='coral', alpha=0.8)

ax4.set_xlabel('Features', fontsize=11)
ax4.set_ylabel('Índice de Shannon', fontsize=11)
ax4.set_title('Comparação: Índice de Shannon em nats vs bits', 
              fontsize=12, fontweight='bold')
ax4.set_xticks(x_pos)
ax4.set_xticklabels(shannon_df.index, rotation=45, ha='right')
ax4.legend(fontsize=10)
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Salvar resultados
shannon_df.to_csv(TAB_DIR / "shannon_index_analysis.csv")
print(f"\n{'='*70}")
print(f"Resultados salvos em: {TAB_DIR / 'shannon_index_analysis.csv'}")
print(f"{'='*70}")

# Análise estatística adicional
print("\n" + "=" * 70)
print("ANÁLISE COMPARATIVA - ÍNDICE DE SHANNON")
print("=" * 70)
print(f"\nFeature mais diversa (maior entropia):")
print(f"  → {shannon_df.index[0]}: {shannon_df.iloc[0]['shannon_index']:.4f} nats")
print(f"\nFeature mais concentrada (menor entropia):")
print(f"  → {shannon_df.index[-1]}: {shannon_df.iloc[-1]['shannon_index']:.4f} nats")
print(f"\nRazão max/min: {shannon_df.iloc[0]['shannon_index'] / shannon_df.iloc[-1]['shannon_index']:.2f}x")
print("=" * 70)

## Cobertura temporal por dataset:
---
Deixar explícito quais fontes cobrem quais períodos, verificando se existe sobreposição suficiente para análises conjuntas. É um slide ótimo para justificar por que alguns modelos/insights só fazem sentido a partir de certo ano (por exemplo, se um índice começa muito depois).

In [ ]:
fig = plt.figure(figsize=(10, 4.8))
ax = fig.add_subplot(111)

timeline = overview.dropna(subset=["min_date", "max_date"]).copy()
timeline = timeline.sort_values("min_date")
y = np.arange(len(timeline))
ax.hlines(y=y, xmin=timeline["min_date"], xmax=timeline["max_date"], label="Janela temporal", color="darkblue")
ax.plot(timeline["min_date"], y, marker="o", linestyle="None", label="Início")
ax.plot(timeline["max_date"], y, marker="o", linestyle="None", label="Fim")
ax.vlines(pd.Timestamp("1994-01-01"), ymin=0, ymax=len(timeline)-1, color="red", linestyle="--", label="Janela temporal de análise")
ax.set_yticks(y)
ax.set_yticklabels(timeline["dataset"])
ax.set_title("Cobertura temporal por dataset (início e fim)")
ax.set_xlabel("Data")
ax.legend()

plt.show()

## Mapa de dados faltantes:
---
Serve para enxergar rapidamente se os faltantes estão concentrados em determinados períodos (ex.: início da série) ou em variáveis específicas. Ajuda em decisões como recorte temporal para a análise, escolha de imputação, ou até a exclusão de variáveis pouco confiáveis.

In [ ]:
num_cols = df_merged.select_dtypes(include=[np.number]).columns.tolist()
heat = df_merged[num_cols].isna().astype(int).to_numpy().T

fig, ax = plt.subplots(figsize=(12, 5.2))
im = ax.imshow(heat, aspect="auto", interpolation="nearest", cmap="binary", vmin=0, vmax=1)
ax.set_title("Mapa de dados faltantes (0 = presente, 1 = ausente) - variáveis numéricas")
ax.set_yticks(np.arange(len(num_cols)))
ax.set_yticklabels(num_cols, fontsize=7)

dates = df_merged.index.to_numpy()
if len(dates) > 0:
    years = pd.DatetimeIndex(dates).year
    tick_idx = np.where((pd.DatetimeIndex(dates).month == 1))[0]
    ax.set_xticks(tick_idx[::2] if len(tick_idx) > 25 else tick_idx)
    ax.set_xticklabels(pd.DatetimeIndex(dates)[ax.get_xticks()].year, rotation=90, fontsize=7)

cbar = fig.colorbar(im, ax=ax, fraction=0.02, pad=0.02, ticks=[0, 1])
cbar.set_label("Dado Faltante", rotation=270, labelpad=15)

In [ ]:
s_urea = df_merged[UREA_COL].astype(float)

fig = plt.figure(figsize=(12, 4.2))
ax = fig.add_subplot(111)
ax.plot(s_urea.index, s_urea.values, label="Ureia (US$/t)")
ax.plot(s_urea.index, s_urea.rolling(12, min_periods=6).mean().values, label="Média móvel 12m")
ax.set_title("Preço da Ureia (Pink Sheet) - nível e tendência (média móvel)")
ax.set_ylabel("US$/t")
ax.legend(loc="best")

plt.show()

In [ ]:
import matplotlib.dates as mdates

feature1 = UREA_COL
feature2 = 'urea_brl'

tmp_df = df_merged[df_merged.index >= pd.to_datetime("1995-01-01")].copy()

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(12, 8))
if feature1 in tmp_df.columns:
    ax1.plot(tmp_df.index, tmp_df[feature1], color='C0', label=feature1)
    ax1.plot(tmp_df.index, tmp_df[feature1].rolling(12, min_periods=6).mean().values, color='gray', label="Média móvel 12m", alpha=0.7)
    ax1.set_ylabel(feature1)
    ax1.legend(loc='upper left')
if feature2 in tmp_df.columns:
    ax2.plot(tmp_df.index, tmp_df[feature2], color='C1', label=feature2)
    ax2.plot(tmp_df.index, tmp_df[feature2].rolling(12, min_periods=6).mean().values, color='gray', label="Média móvel 12m", alpha=0.7)
    ax2.set_ylabel(feature2)
    ax2.legend(loc='upper left')
if feature1 in tmp_df.columns and feature2 in tmp_df.columns:
    ax3.plot(tmp_df.index, tmp_df[feature1], color='C0', label=feature1)
    ax3.plot(tmp_df.index, tmp_df[feature2], color='C1', label=feature2)
    ax3.legend(loc='upper left')
    ax3.set_ylabel('preço/t')

    years = pd.date_range(tmp_df.index.values.min(), tmp_df.index.values.max(), freq="YS")
    ax3.set_xticks(years)
    ax3.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
    ax3.tick_params(axis="x", rotation=60)

ax2.set_xlabel('Date')
fig.suptitle(f'Ureia em dólares e em reais ao longo do tempo (30 anos)', fontsize=16)
fig.tight_layout(rect=[0, 0.03, 1, 0.99])
plt.show()

In [ ]:
import matplotlib.dates as mdates

feature1 = 'urea_usd'
feature2 = 'urea_brl'

tmp_df = df_merged[df_merged.index >= pd.to_datetime("1995-01-01")].copy()

# Normalizar com z-score
tmp_df['urea_usd_norm'] = (tmp_df[feature1] - tmp_df[feature1].mean()) / tmp_df[feature1].std()
tmp_df['urea_brl_norm'] = (tmp_df[feature2] - tmp_df[feature2].mean()) / tmp_df[feature2].std()

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(12, 8))

# Gráfico 1: Séries normalizadas
ax1.plot(tmp_df.index, tmp_df['urea_usd_norm'], color='C0', label='urea_usd (normalizado)', linewidth=2)
ax1.plot(tmp_df.index, tmp_df['urea_brl_norm'], color='C1', label='urea_brl (normalizado)', linewidth=2)
ax1.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax1.set_ylabel('Z-score')
ax1.set_title('Ureia USD vs BRL - Séries Normalizadas (Z-score)')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# Gráfico 2: Escala original (para referência)
ax2_right = ax2.twinx()
ax2.plot(tmp_df.index, tmp_df[feature1], color='C0', label='urea_usd', linewidth=2)
ax2_right.plot(tmp_df.index, tmp_df[feature2], color='C1', label='urea_brl', linewidth=2)
ax2.set_ylabel('US$/t', color='C0')
ax2_right.set_ylabel('R$/t', color='C1')
ax2.tick_params(axis='y', labelcolor='C0')
ax2_right.tick_params(axis='y', labelcolor='C1')
ax2.set_title('Ureia USD vs BRL - Escala Original')
ax2.legend(loc='upper left', fontsize=8)
ax2_right.legend(loc='upper right', fontsize=8)
ax2.grid(True, alpha=0.3)

years = pd.date_range(tmp_df.index.values.min(), tmp_df.index.values.max(), freq="YS")
ax2.set_xticks(years)
ax2.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
ax2.tick_params(axis="x", rotation=60)

fig.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
tmp_df['urea_usd_norm'] = scaler.fit_transform(tmp_df[[feature1]])
tmp_df['urea_brl_norm'] = scaler.fit_transform(tmp_df[[feature2]])

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(tmp_df.index, tmp_df['urea_usd_norm'], color='C0', label='urea_usd (0-1)', linewidth=2)
ax.plot(tmp_df.index, tmp_df['urea_brl_norm'], color='C1', label='urea_brl (0-1)', linewidth=2)
ax.set_ylabel('Valor Normalizado (0-1)')
ax.set_title('Ureia USD vs BRL - Min-Max Normalization')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(tmp_df.index, tmp_df[feature1].pct_change() * 100, color='C0', label='urea_usd (% change)', linewidth=1.5, alpha=0.8)
ax.plot(tmp_df.index, tmp_df[feature2].pct_change() * 100, color='C1', label='urea_brl (% change)', linewidth=1.5, alpha=0.8)
ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.set_ylabel('Variação (%)')
ax.set_title('Ureia USD vs BRL - Variação Mensal (%)')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
s_urea = df_merged[UREA_COL].astype(float)

ret = s_urea.pct_change() * 100
vol12 = ret.rolling(12, min_periods=6).std()

fig = plt.figure(figsize=(12, 4.2))
ax = fig.add_subplot(111)
ax.plot(ret.index, ret.values, label="Variação % mensal")
ax.plot(vol12.index, vol12.values, label="Volatilidade 12m (desvio-padrão)")
ax.set_title("Ureia (USD) - variação percentual mensal x volatilidade móvel")
ax.set_ylabel("%")
ax.legend(loc="best")

plt.show()

In [ ]:
s_urea = s_urea[s_urea.index >= pd.to_datetime("1995-01-01")]

tmp = pd.DataFrame({"date": s_urea.index, "urea": s_urea.values})
tmp["month"] = tmp["date"].dt.month
data_by_month = [tmp.loc[tmp["month"] == m, "urea"].dropna().values for m in range(1, 13)]

fig = plt.figure(figsize=(10.8, 4.2))
ax = fig.add_subplot(111)
ax.boxplot(data_by_month, labels=[str(m) for m in range(1, 13)], showfliers=False)
ax.set_title("Boxplot - distribuição do preço da Ureia por mês (30 anos - 1995 a 2025)")
ax.set_xlabel("Mês")
ax.set_ylabel("US$/t")

plt.show()

In [ ]:
tmp = pd.DataFrame({"date": s_urea.index, "urea": s_urea.values})
tmp["month"] = tmp["date"].dt.month
tmp["year"] = tmp["date"].dt.year
tmp["year_group"] = (tmp["year"] // 10) * 10

year_groups = sorted(tmp["year_group"].unique())
year_labels = [f"{yr}-{yr+9}" if yr + 9 <= tmp["year"].max() else f"{yr}-{tmp['year'].max()}" for yr in year_groups]

fig, ax = plt.subplots(figsize=(16, 6))

# Posições para os boxplots (mês, depois período)
positions = []
data_to_plot = []
colors_list = []
month_to_positions = {m: [] for m in range(1, 13)}  # Mapear mês para posições
pos = 1

color_palette = ['lightblue', 'lightgreen', 'lightpink']

for month in range(1, 13):
    for idx, year_group in enumerate(year_groups):
        data = tmp.loc[(tmp["year_group"] == year_group) & (tmp["month"] == month), "urea"].dropna().values
        if len(data) > 0:
            data_to_plot.append(data)
            positions.append(pos)
            colors_list.append(color_palette[idx % len(color_palette)])
            month_to_positions[month].append(pos)  # Guardar posição para este mês
            pos += 1
    pos += 1  # Espaço entre meses

bp = ax.boxplot(data_to_plot, positions=positions, widths=0.6, patch_artist=True, showfliers=False)

# Colorir os boxplots
for patch, color in zip(bp['boxes'], colors_list):
    patch.set_facecolor(color)

# Adicionar labels para os meses
month_positions = []
month_labels = []
for month in range(1, 13):
    if month_to_positions[month]:  # Se há dados para este mês
        avg_pos = np.mean(month_to_positions[month])
        month_positions.append(avg_pos)
        month_labels.append(str(month))

ax.set_xticks(month_positions)
ax.set_xticklabels(month_labels)
ax.set_ylabel("US$/t")
ax.set_title("Boxplot - Distribuição do preço da Ureia por mês e período de 10 anos", fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Legenda com cores para períodos
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=color_palette[i % len(color_palette)], label=year_labels[i]) 
                   for i in range(len(year_groups))]
ax.legend(handles=legend_elements, loc='upper left', title='Período', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import STL
import matplotlib.dates as mdates

stl_series = s_urea.dropna()
stl = STL(stl_series, period=12, robust=True).fit()

translations = {
    "trend": "tendência",
    "seasonal": "sazonalidade",
    "resid": "residual",
}

for comp_name, comp_series in [
    ("trend", stl.trend),
    ("seasonal", stl.seasonal),
    ("resid", stl.resid),
]:
    fig, ax = plt.subplots(figsize=(12, 4.0))
    ax.plot(comp_series.index, comp_series.values, linewidth=1.5)
    ax.set_title(f"Decomposição STL - componente: {translations[comp_name]}", fontsize=13, fontweight='bold')
    ax.set_ylabel("US$/t", fontsize=11)
    ax.set_xlabel("Ano", fontsize=11)
    
    # Configurar anos no eixo X
    ax.xaxis.set_major_locator(mdates.YearLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    
    # Rotação e alinhamento
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right', fontsize=9)
    
    # Grid para melhor leitura
    ax.grid(True, alpha=0.3, linestyle='--')
    
    fig.tight_layout()

plt.show()

In [ ]:
stl.plot()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import calendar

# 1. Extrair a série sazonal do seu objeto STL já treinado
seasonal = stl.seasonal

# 2. Filtrar apenas os últimos 5 anos de dados
# Pega a última data disponível e subtrai 5 anos
data_corte = seasonal.index.max() - pd.DateOffset(years=5)
seasonal_recent = seasonal[seasonal.index > data_corte]

# 3. Agrupar por mês e calcular a média da sazonalidade para cada mês
# O index.month retorna 1 para Jan, 2 para Fev, etc.
monthly_avg = seasonal_recent.groupby(seasonal_recent.index.month).mean()

# Criar nomes dos meses para o eixo X
month_names = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 
               'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

# 4. Plotar o Gráfico de Barras
fig, ax = plt.subplots(figsize=(10, 5))

# Criar cores condicionais: Verde para preço baixo (compra), Vermelho para preço alto
colors = ['green' if x < 0 else 'red' for x in monthly_avg.values]

bars = ax.bar(month_names, monthly_avg.values, color=colors, alpha=0.7, edgecolor='black')

# Estilização
ax.set_title("Sazonalidade média mensal (últimos 5 anos)", fontsize=14)
ax.set_ylabel("Impacto no Preço (US$/t)", fontsize=12)
ax.set_xlabel("Mês", fontsize=12)
ax.axhline(0, color='black', linewidth=1, linestyle='-') # Linha zero
ax.grid(axis='y', linestyle='--', alpha=0.5)

# Adicionar o valor exato em cima/baixo das barras
for bar in bars:
    height = bar.get_height()
    offset = 2 if height > 0 else -2  # Ajuste da posição do texto
    ax.text(bar.get_x() + bar.get_width()/2., height + offset,
            f'{height:.1f}',
            ha='center', va='bottom' if height > 0 else 'top', 
            fontsize=10, fontweight='bold', color='black')

ax.set_ylim(-60, 60)

plt.tight_layout()
plt.show()

In [ ]:
key_cols = [
    "urea_usd",
    "natural_gas_usd",
    "crude_oil_usd",
    "usdbrl",
    "gscpi",
    "oni",
    "gpr",
    "dap_usd",
    "potassium_usd",
    "maize_usd",
    "wheat_usd",
    "soybeans_usd",
]
key_cols = [c for c in key_cols if c in df_merged.columns]
corr = df_merged[key_cols].corr(numeric_only=True)

fig = plt.figure(figsize=(9.5, 8.0))
ax = fig.add_subplot(111)
im = ax.imshow(corr.values, interpolation="nearest", aspect="auto", cmap="RdYlGn", vmin=-1, vmax=1)
ax.set_title("Matriz de correlação geral")
ax.set_xticks(np.arange(len(key_cols)))
ax.set_yticks(np.arange(len(key_cols)))
ax.set_xticklabels(key_cols, rotation=90, fontsize=7)
ax.set_yticklabels(key_cols, fontsize=7)
fig.colorbar(im, ax=ax, fraction=0.03, pad=0.02)

In [ ]:
df_merged.columns

In [ ]:
import re

def lagged_corr(target: pd.Series, feature: pd.Series, max_lag: int = 24):
    # corr(target_t, feature_{t-lag}) for lag in [-max_lag, +max_lag]
    lags = np.arange(-max_lag, max_lag + 1)
    out = []
    for lag in lags:
        shifted = feature.shift(lag)
        d = pd.concat([target, shifted], axis=1).dropna()
        out.append(d.iloc[:, 0].corr(d.iloc[:, 1]) if len(d) > 5 else np.nan)
    return pd.Series(out, index=lags, name="corr")

translations = {
    "natural_gas_usd": "Gás Natural (US$/MMBtu)",
    "crude_oil_usd": "Petróleo Bruto (US$/bbl)",
    "usdbrl": "Taxa de Câmbio USD/BRL",
    "gscpi": "Global Supply Chain Pressure Index (GSCPI)",
    "oni": "Oceanic Niño Index (ONI)",
    "gpr": "Índice de Risco Geopolítico Global (GPR)",
}

lag_features = ["natural_gas_usd", "crude_oil_usd", "usdbrl", "gscpi", "oni", "gpr"]
lag_summ = []

for feat in lag_features:
    if feat not in df_merged.columns:
        continue
    lc = lagged_corr(s_urea, df_merged[feat].astype(float), max_lag=18)
    best_lag = int(lc.abs().idxmax()) if lc.dropna().size else np.nan
    best_corr = float(lc.loc[best_lag]) if not np.isnan(best_lag) else np.nan
    lag_summ.append({"feature": feat, "best_lag_months": best_lag, "corr_at_best_lag": best_corr})
    lc.to_csv(TAB_DIR / f"09_lagged_corr_{re.sub(r'[^a-zA-Z0-9]+','_',feat).lower()}.csv")

    fig = plt.figure(figsize=(10.5, 4.0))
    ax = fig.add_subplot(111)
    ax.plot(lc.index, lc.values)
    ax.axhline(0, linewidth=1, color="gray", linestyle="--")
    ax.set_title(f"Correlação defasada: Ureia vs {translations.get(feat, feat)} (lag em meses)")
    ax.set_xlabel("Lag (meses). Positivo = feature atrasada; Negativo = feature adiantada")
    ax.set_ylabel("Correlação")

plt.show()

In [ ]:
df_merged[["urea_usd", "crude_oil_usd", "natural_gas_usd", "usdbrl", "gscpi", "gpr"]].corr()

In [ ]:
from scipy.stats import entropy

entropy

In [ ]:
def rolling_corr(a: pd.Series, b: pd.Series, window: int = 12):
    d = pd.concat([a, b], axis=1).dropna()
    return d.iloc[:, 0].rolling(window).corr(d.iloc[:, 1])

for feat in ["natural_gas_usd", "crude_oil_usd", "usdbrl", "gscpi", "gpr"]:
    if feat not in df_merged.columns:
        continue
    rc = rolling_corr(s_urea, df_merged[feat].astype(float), window=24)
    fig = plt.figure(figsize=(12, 4.0))
    ax = fig.add_subplot(111)
    ax.plot(rc.index, rc.values)
    ax.axhline(0, linewidth=1, color="gray", linestyle="--")
    ax.set_title(f"Correlação móvel (24m): Ureia vs {translations.get(feat, feat)}")
    ax.set_ylabel("Correlação")

    plt.savefig(f"./tmp_imgs/rolling_corr_24m_{feat}.png")

    rc = rolling_corr(s_urea, df_merged[feat].astype(float), window=12)
    fig = plt.figure(figsize=(12, 4.0))
    ax = fig.add_subplot(111)
    ax.plot(rc.index, rc.values)
    ax.axhline(0, linewidth=1, color="gray", linestyle="--")
    ax.set_title(f"Correlação móvel (12m): Ureia vs {translations.get(feat, feat)}")
    ax.set_ylabel("Correlação")

    plt.savefig(f"./tmp_imgs/rolling_corr_12m_{feat}.png")

    rc = rolling_corr(s_urea, df_merged[feat].astype(float), window=6)
    fig = plt.figure(figsize=(12, 4.0))
    ax = fig.add_subplot(111)
    ax.plot(rc.index, rc.values)
    ax.axhline(0, linewidth=1, color="gray", linestyle="--")
    ax.set_title(f"Correlação móvel (6m): Ureia vs {translations.get(feat, feat)}")
    ax.set_ylabel("Correlação")

    plt.savefig(f"./tmp_imgs/rolling_corr_6m_{feat}.png")

In [ ]:
norm_cols = ["urea_usd", "natural_gas_usd", "crude_oil_usd", "usdbrl", "gscpi", "gpr"]
norm_cols = [c for c in norm_cols if c in df_merged.columns]
norm_df = df_merged[norm_cols].apply(zscore)

fig = plt.figure(figsize=(12, 4.6))
ax = fig.add_subplot(111)
for c in norm_cols:
    ax.plot(norm_df.index, norm_df[c].values, label=translations.get(c, c))
ax.set_title("Séries normalizadas (z-score) para comparar movimentos (sem escala)")
ax.set_ylabel("z-score")
ax.legend(loc="best", fontsize=7)

In [ ]:
if "usdbrl" in df_merged.columns:
    urea_brl = df_merged["urea_usd"].astype(float) * df_merged["usdbrl"].astype(float)
    df_merged["urea_brl"] = urea_brl

    fig = plt.figure(figsize=(12, 4.2))
    ax = fig.add_subplot(111)
    ax.plot(urea_brl.index, urea_brl.values, label="Ureia (R$/t)")
    ax.plot(urea_brl.index, urea_brl.rolling(12, min_periods=6).mean().values, label="Média móvel 12m")
    ax.set_title("Custo implícito no Brasil: Ureia (US$/t) * câmbio (USD/BRL)")
    ax.set_ylabel("R$/t")
    ax.legend(loc="best")

## Marcação dos Eventos:
---

In [ ]:
events = df_events.dropna(subset=["date", "event"]).copy()
events = events.sort_values("date")

# Compute event-month return
event_impact = []
for _, row in events.iterrows():
    d = row["date"]
    if d not in df_merged.index:
        continue
    
    # Usar .iloc[0] para pegar o primeiro valor se houver duplicatas
    if d in ret.index:
        ret_val = ret.loc[d]
        r = float(ret_val.iloc[0]) if isinstance(ret_val, pd.Series) else float(ret_val)
    else:
        r = np.nan
    
    if d in s_urea.index:
        price_val = s_urea.loc[d]
        p = float(price_val.iloc[0]) if isinstance(price_val, pd.Series) else float(price_val)
    else:
        p = np.nan
    
    event_impact.append({"date": d, "event": row["event"], "urea_usd_mt": p, "urea_mom_pct": r})

event_impact_df = pd.DataFrame(event_impact).sort_values("date")
event_impact_df.to_csv(TAB_DIR / "13_event_impact_table.csv", index=False)

# Plot with markers for events
fig = plt.figure(figsize=(12, 4.5))
ax = fig.add_subplot(111)
ax.plot(s_urea.index, s_urea.values, label="urea_usd", linewidth=1.5)

# Obter valores de ureia para as datas dos eventos
ev_dates = event_impact_df["date"].values
ev_vals = event_impact_df["urea_usd_mt"].values

# Plotar marcadores apenas onde existem valores válidos
valid_mask = ~np.isnan(ev_vals)
ax.scatter(ev_dates[valid_mask], ev_vals[valid_mask], 
           label="Eventos históricos", s=50, color='red', 
           marker='o', zorder=5, alpha=0.7)

ax.set_title("Ureia com marcação de meses com eventos históricos (main_events)", 
             fontsize=13, fontweight='bold')
ax.set_ylabel("US$/t", fontsize=11)
ax.set_xlabel("Data", fontsize=11)
ax.legend(loc="best", fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
trade = df_trade.copy()
trade["flowDesc"] = trade["flowDesc"].astype(str)

def plot_trade_country(country_prefix: str, flow: str, y_col: str, title: str, filename: str):
    col = f"{country_prefix}_{y_col}"
    if col not in trade.columns:
        return
    d = trade.loc[trade["flowDesc"] == flow, ["date", col]].copy()
    d = d.dropna().sort_values("date").set_index("date")
    # join urea price for overlapping range
    joined = d.join(s_urea.rename("urea_price"), how="left")
    fig = plt.figure(figsize=(12, 4.3))
    ax = fig.add_subplot(111)
    ax.plot(joined.index, joined[col].values, label=f"{country_prefix} {flow} {y_col}")
    ax2 = ax.twinx()
    ax2.plot(joined.index, joined["urea_price"].values, label="urea_usd")
    ax.set_title(title)
    ax.set_ylabel(y_col)
    ax2.set_ylabel("US$/t")
    # combined legend
    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc="best", fontsize=7)
    # savefig(fig, filename)

plot_trade_country("BRA", "Import", "tonnes", "Brasil: Importação (toneladas) vs preço da ureia", "15_bra_import_tonnes_vs_price.png")
plot_trade_country("BRA", "Export", "tonnes", "Brasil: Exportação (toneladas) vs preço da ureia", "16_bra_export_tonnes_vs_price.png")
plot_trade_country("CHN", "Export", "tonnes", "China: Exportação (toneladas) vs preço da ureia", "17_chn_export_tonnes_vs_price.png")
plot_trade_country("CHN", "Import", "tonnes", "China: Importação (toneladas) vs preço da ureia", "18_chn_import_tonnes_vs_price.png")

In [ ]:
fig = plt.figure(figsize=(16, 6))
ax = fig.add_subplot(111)
ax.plot(df_merged.index, df_merged["crude_oil_usd"], label="Crude Oil (US$/bbl)")
ax.plot(df_merged.index, df_merged["urea_usd"], label="Ureia (US$/t)")
ax.set_ylabel("US$/t")
ax.legend(loc="best")

plt.show()

In [ ]:
import statsmodels.api as sm

tmp = df_merged.drop(columns=["period", "urea_brl"])
tmp = tmp[['urea_usd', 'natural_gas_usd', 'crude_oil_usd', 'maize_usd',
       'wheat_usd', 'soybeans_usd', 'dap_usd', 'potassium_usd', 'usdbrl', 'gpr']]

cols_num = tmp.select_dtypes(include=[np.number]).columns
cols_x = cols_num.drop("urea_usd")  ## Target
 
X = tmp[cols_x].fillna(0)
X = sm.add_constant(X)
 
y = tmp["urea_usd"]
 
model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from statsmodels.stats.diagnostic import het_breuschpagan, acorr_breusch_godfrey
# from statsmodels.stats.stattools import durbin_watson

# resid = results.resid
# print("Durbin-Watson:", durbin_watson(resid))

# bp = het_breuschpagan(resid, model.model.exog)
# print("Breusch-Pagan p-value:", bp[1])

# bg = acorr_breusch_godfrey(model, nlags=12)
# print("Breusch-Godfrey p-value:", bg[1])

# print("Omnibus:", model.omnibus, "Jarque-Bera p:", model.jarque_bera[1])

# # VIF (remover 'const' antes)
# X_no_const = X.drop(columns=["const"])
# vif = pd.Series(
#     [variance_inflation_factor(X_no_const.values, i) for i in range(X_no_const.shape[1])],
#     index=X_no_const.columns
# ).sort_values(ascending=False)
# print("VIF:\n", vif)

# # Ajustes com covariância robusta
# print("Robust (HC3):")
# print(model.get_robustcov_results(cov_type="HC3").summary())

# print("Newey-West (HAC, 12 lags):")
# print(model.get_robustcov_results(cov_type="HAC", maxlags=12).summary())

# # Exemplo rápido: regressão em diferenças (para lidar com não-estacionaridade)
# y_d = y.diff().dropna()
# X_d = X.loc[y_d.index].diff().dropna()
# X_d = sm.add_constant(X_d)
# model_diff = sm.OLS(y_d, X_d).fit()
# print("Diff model summary:")
# print(model_diff.summary())

In [ ]:
# # ...existing code...
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from statsmodels.stats.diagnostic import het_breuschpagan, acorr_breusch_godfrey
# from statsmodels.stats.stattools import durbin_watson, jarque_bera

# # usar o objeto 'model' (fitted) e não 'results'
# resid = model.resid

# print("Durbin-Watson:", durbin_watson(resid))

# bp = het_breuschpagan(resid, model.model.exog)
# print("Breusch-Pagan p-value:", bp[1])

# bg = acorr_breusch_godfrey(model, nlags=12)
# print("Breusch-Godfrey p-value:", bg[1])

# # testes de normalidade (função Jarque-Bera)
# jb_stat, jb_p, jb_skew, jb_kurt = jarque_bera(resid)
# print("Jarque-Bera stat:", jb_stat, "p-value:", jb_p, "skew:", jb_skew, "kurtosis:", jb_kurt)

# # VIF (remover 'const')
# X_no_const = X.drop(columns=["const"])
# vif = pd.Series(
#     [variance_inflation_factor(X_no_const.values, i) for i in range(X_no_const.shape[1])],
#     index=X_no_const.columns
# ).sort_values(ascending=False)
# print("VIF:\n", vif)

# # covariância robusta / Newey-West
# print("Robust (HC3):")
# print(model.get_robustcov_results(cov_type="HC3").summary())

# print("Newey-West (HAC, 12 lags):")
# print(model.get_robustcov_results(cov_type="HAC", maxlags=12).summary())

# # regressão em diferenças (exemplo para não-estacionaridade)
# y_d = y.diff().dropna()
# X_d = X.loc[y_d.index].diff().dropna()
# X_d = sm.add_constant(X_d)
# model_diff = sm.OLS(y_d, X_d).fit()
# print("Diff model summary:")
# print(model_diff.summary())
# # ...existing code...

In [ ]:
# regressão em diferenças (exemplo para não-estacionaridade)
y_d = y.diff().dropna()

# remover 'const' antes de diferenciar para evitar colunas constantes (todos zeros)
X_no_const = X.drop(columns=["const"], errors="ignore")

# diferenciar e dropar NaNs
X_d = X_no_const.diff().dropna()

# alinhar índices (interseção) para garantir igualdade de comprimento e ordem
common_idx = y_d.index.intersection(X_d.index)
y_d = y_d.loc[common_idx]
X_d = X_d.loc[common_idx]

# adicionar constante e ajustar
X_d = sm.add_constant(X_d)
model_diff = sm.OLS(y_d, X_d).fit()
print("Diff model summary:")
print(model_diff.summary())

## Gráfico 5: Ureia vs Gás Natural (Níveis + Correlação Defasada)
---
Ureia é intensiva em energia. Em vários períodos, energia antecede movimentos de ureia, o que sugere que gás pode ser um sinal de alerta antecipado.

In [ ]:
# Gráfico 1: Série temporal - Ureia vs Gás Natural (escala dupla)
fig, ax1 = plt.subplots(figsize=(12, 5.5))

# Eixo esquerdo: Ureia
s_ng = df_merged["natural_gas_usd"].astype(float)
ax1.plot(s_urea.index, s_urea.values, color='C0', linewidth=2, label='Ureia (US$/t)')
ax1.set_xlabel('Data')
ax1.set_ylabel('Ureia (US$/t)', color='C0')
ax1.tick_params(axis='y', labelcolor='C0')
ax1.grid(True, alpha=0.3)

# Eixo direito: Gás Natural
ax2 = ax1.twinx()
ax2.plot(s_ng.index, s_ng.values, color='C1', linewidth=2, label='Gás Natural (US$/mmbtu)')
ax2.set_ylabel('Gás Natural (US$/mmbtu)', color='C1')
ax2.tick_params(axis='y', labelcolor='C1')

fig.suptitle('Ureia vs Gás Natural - Série Temporal (1990-2025)', fontsize=14, fontweight='bold')

# Legenda unificada
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=10)

# savefig(fig, '20_urea_vs_gas_timeseries.png')

plt.show()


In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 5.5))

# Eixo esquerdo: Ureia
s_ng = df_merged["crude_oil_usd"].astype(float)
ax1.plot(s_urea.index, s_urea.values, color='C0', linewidth=2, label='Ureia (US$/t)')
ax1.set_xlabel('Data')
ax1.set_ylabel('Ureia (US$/t)', color='C0')
ax1.tick_params(axis='y', labelcolor='C0')
ax1.grid(True, alpha=0.3)

# Eixo direito: Gás Natural
ax2 = ax1.twinx()
ax2.plot(s_ng.index, s_ng.values, color='C1', linewidth=2, label='Petróleo Bruto (US$/bbl)')
ax2.set_ylabel('Petróleo Bruto (US$/bbl)', color='C1')
ax2.tick_params(axis='y', labelcolor='C1')

fig.suptitle('Ureia vs Petróleo Bruto - Série Temporal (1990-2025)', fontsize=14, fontweight='bold')

# Legenda unificada
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=10)

# savefig(fig, '20_urea_vs_gas_timeseries.png')

plt.show()


In [ ]:
# Calcular correlação com lags de -18 a +18 meses
lc_gas = lagged_corr(s_urea, s_ng, max_lag=18)

# Identificar o melhor lag (máxima correlação em valor absoluto)
best_lag_gas = int(lc_gas.abs().idxmax())
best_corr_gas = float(lc_gas.loc[best_lag_gas])

fig, ax = plt.subplots(figsize=(11, 5))
ax.plot(lc_gas.index, lc_gas.values, marker='o', markersize=5, linewidth=2, color='C1')
ax.axhline(0, color='gray', linestyle='--', linewidth=1, alpha=0.7)
ax.axvline(best_lag_gas, color='red', linestyle='--', linewidth=2, alpha=0.7, label=f'Melhor lag: {best_lag_gas} meses (r={best_corr_gas:.3f})')

ax.set_xlabel('Lag (meses)\nPositivo = Gás antecede Ureia | Negativo = Ureia antecede Gás', fontsize=10)
ax.set_ylabel('Correlação', fontsize=11)
ax.set_title('Correlação Defasada: Ureia vs Gás Natural\n(Defasagem histórica de energía como preditor)', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(loc='best', fontsize=10)

savefig(fig, '21_urea_gas_lagged_correlation.png')

# Resumo interpretativo
print("=" * 70)
print("ANÁLISE: UREIA vs GÁS NATURAL (CORRELAÇÃO DEFASADA)")
print("=" * 70)
print(f"\nMelhor lag: {best_lag_gas} meses")
print(f"Correlação no melhor lag: {best_corr_gas:.4f}")
if best_lag_gas > 0:
    print(f"→ Interpretação: Gás Natural ANTECEDE Ureia por ~{best_lag_gas} mês(es)")
    print("  Implicação: Mudanças em preço de gás podem ser sinal antecipado para ureia")
elif best_lag_gas < 0:
    print(f"→ Interpretação: Ureia ANTECEDE Gás Natural por ~{abs(best_lag_gas)} mês(es)")
    print("  Implicação: Mudanças em preço de ureia precedem movimento de gás")
else:
    print("→ Interpretação: Máxima correlação ocorre SEM defasagem")
    print("  Implicação: Movimentos são contemporâneos")

# Top 5 lags positivos (gás antecede)
top_lags = lc_gas.abs().nlargest(5)
print("\n--- Top 5 Lags (por valor absoluto de correlação) ---")
for lag, corr_val in top_lags.items():
    actual_corr = float(lc_gas.loc[lag])
    print(f"  Lag {lag:3d}: corr = {actual_corr:7.4f} (abs: {abs(actual_corr):.4f})")
print("=" * 70)


In [ ]:
# 3) ECM: Δy ~ ΔX + lag(residuals_levels)
y_d = y.diff().dropna()
X_no_const = X.drop(columns=["const"], errors="ignore")
X_d = X_no_const.diff().dropna()

# lag dos resíduos (não restringir ainda)
res_lag = model.resid.shift(1)

# concatenar e alinhar antes de dropar NaNs
ecm_df = pd.concat([X_d, res_lag.drop(pd.to_datetime('1995-01-01'), axis=0).rename("ecm")], axis=1)

# intersecção de índices e dropar linhas com NaNs
common_idx = y_d.index.intersection(ecm_df.index)
y_ecm = y_d.loc[common_idx]
X_ecm = ecm_df.loc[common_idx].dropna()

# re-alinhar no caso de dropna ter removido mais linhas
common_idx2 = y_ecm.index.intersection(X_ecm.index)
y_ecm = y_ecm.loc[common_idx2]
X_ecm = X_ecm.loc[common_idx2]

print("ECM shapes:", y_ecm.shape, X_ecm.shape)  # debug rápido

X_ecm = sm.add_constant(X_ecm)
model_ecm = sm.OLS(y_ecm, X_ecm).fit()
print(model_ecm.summary())

In [ ]:
import seaborn as sns
from scipy.stats import pearsonr
from statsmodels.tsa.stattools import ccf

# ============================================================================
# 1. HEATMAP DE CORRELAÇÃO - NÍVEIS
# ============================================================================
print("="*70)
print("1. MATRIZ DE CORRELAÇÃO - NÍVEIS")
print("="*70)

key_vars = ['urea_usd', 'natural_gas_usd', 'crude_oil_usd', 'usdbrl',
            'gscpi', 'oni', 'gpr', 'dap_usd', 'potassium_usd', 'maize_usd', 'wheat_usd', 'soybeans_usd']
key_vars = [v for v in key_vars if v in df_merged.columns]

corr_levels = df_merged[key_vars].corr()

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr_levels, annot=True, fmt='.3f', cmap='RdYlGn', center=0,
            vmin=-1, vmax=1, square=True, linewidths=0.5, 
            cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Matriz de Correlação - Níveis (Pearson)', 
             fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
# plt.savefig(FIG_DIR / '30_correlation_heatmap_levels.png', dpi=160)
plt.show()

# ============================================================================
# 2. HEATMAP DE CORRELAÇÃO - VARIAÇÕES MOM (%)
# ============================================================================
print("\n" + "="*70)
print("2. MATRIZ DE CORRELAÇÃO - VARIAÇÕES MOM (%)")
print("="*70)

df_mom = df_merged[key_vars].pct_change() * 100
df_mom.columns = [f"{c}_mom" for c in df_mom.columns]
corr_mom = df_mom.corr()

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr_mom, annot=True, fmt='.3f', cmap='RdYlGn', center=0,
            vmin=-1, vmax=1, square=True, linewidths=0.5,
            cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Matriz de Correlação - Variações MoM (%)', 
             fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
# plt.savefig(FIG_DIR / '31_correlation_heatmap_mom.png', dpi=160)
plt.show()

# ============================================================================
# 3. HEATMAP DE CORRELAÇÃO - VARIAÇÕES YOY (%)
# ============================================================================
print("\n" + "="*70)
print("3. MATRIZ DE CORRELAÇÃO - VARIAÇÕES YOY (%)")
print("="*70)

df_yoy = df_merged[key_vars].pct_change(12) * 100
df_yoy.columns = [f"{c}_yoy" for c in df_yoy.columns]
corr_yoy = df_yoy.corr()

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr_yoy, annot=True, fmt='.3f', cmap='RdYlGn', center=0,
            vmin=-1, vmax=1, square=True, linewidths=0.5,
            cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Matriz de Correlação - Variações YoY (%)', 
             fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
# plt.savefig(FIG_DIR / '32_correlation_heatmap_yoy.png', dpi=160)
plt.show()

# ============================================================================
# 4. SCATTER PLOTS COM LINHA DE TENDÊNCIA
# ============================================================================
print("\n" + "="*70)
print("4. SCATTER PLOTS - UREIA vs VARIÁVEIS CHAVE")
print("="*70)

scatter_vars = ['natural_gas_usd', 'crude_oil_usd', 'usdbrl', 'gscpi']
scatter_vars = [v for v in scatter_vars if v in df_merged.columns]

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, var in enumerate(scatter_vars):
    ax = axes[idx]
    
    # Dados válidos
    data = df_merged[['urea_usd', var]].dropna()
    x = data[var].values
    y = data['urea_usd'].values
    
    # Scatter
    ax.scatter(x, y, alpha=0.5, s=20, edgecolors='k', linewidths=0.5)
    
    # Linha de tendência
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    x_line = np.linspace(x.min(), x.max(), 100)
    ax.plot(x_line, p(x_line), "r--", linewidth=2, label=f'y={z[0]:.2f}x+{z[1]:.2f}')
    
    # Correlação
    corr, pval = pearsonr(x, y)
    
    ax.set_xlabel(translations.get(var, var), fontsize=11)
    ax.set_ylabel('Ureia (US$/t)', fontsize=11)
    ax.set_title(f'Ureia vs {translations.get(var, var)}\nCorr={corr:.3f}, p={pval:.4f}',
                 fontsize=12, fontweight='bold')
    ax.legend(loc='best', fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
# plt.savefig(FIG_DIR / '33_scatter_plots_urea_vs_key_vars.png', dpi=160)
plt.show()

# ============================================================================
# 5. SCATTER POR REGIME (PRÉ/PÓS EVENTOS IMPORTANTES)
# ============================================================================
print("\n" + "="*70)
print("5. SCATTER POR REGIME TEMPORAL")
print("="*70)

# Definir regimes baseados em eventos importantes
regimes = {
    'Pre-COVID (< 2020)': df_merged.index < pd.Timestamp('2020-01-01'),
    'COVID (2020-2021)': (df_merged.index >= pd.Timestamp('2020-01-01')) & 
                         (df_merged.index < pd.Timestamp('2022-01-01')),
    'Guerra Ucrânia (2022+)': df_merged.index >= pd.Timestamp('2022-01-01')
}

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Ureia vs Gás Natural
ax = axes[0]
colors = ['blue', 'orange', 'red']
for idx, (regime_name, mask) in enumerate(regimes.items()):
    data = df_merged.loc[mask, ['urea_usd', 'natural_gas_usd']].dropna()
    if len(data) > 0:
        ax.scatter(data['natural_gas_usd'], data['urea_usd'], 
                  alpha=0.6, s=30, label=regime_name, color=colors[idx])

ax.set_xlabel('Gás Natural (US$/MMBtu)', fontsize=11)
ax.set_ylabel('Ureia (US$/t)', fontsize=11)
ax.set_title('Ureia vs Gás Natural - Por Regime', fontsize=13, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

# Plot 2: Ureia vs USD/BRL
ax = axes[1]
for idx, (regime_name, mask) in enumerate(regimes.items()):
    data = df_merged.loc[mask, ['urea_usd', 'usdbrl']].dropna()
    if len(data) > 0:
        ax.scatter(data['usdbrl'], data['urea_usd'], 
                  alpha=0.6, s=30, label=regime_name, color=colors[idx])

ax.set_xlabel('USD/BRL', fontsize=11)
ax.set_ylabel('Ureia (US$/t)', fontsize=11)
ax.set_title('Ureia vs Câmbio - Por Regime', fontsize=13, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
# plt.savefig(FIG_DIR / '34_scatter_by_regime.png', dpi=160)
plt.show()

# ============================================================================
# 6. CROSS-CORRELATION FUNCTION (CCF)
# ============================================================================
print("\n" + "="*70)
print("6. CROSS-CORRELATION FUNCTION (CCF)")
print("="*70)

ccf_vars = ['natural_gas_usd', 'crude_oil_usd', 'usdbrl', 'gscpi']
ccf_vars = [v for v in ccf_vars if v in df_merged.columns]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

max_lag = 12

for idx, var in enumerate(ccf_vars):
    ax = axes[idx]
    
    # Dados válidos
    data = df_merged[['urea_usd', var]].dropna()
    y = data['urea_usd'].values
    x = data[var].values
    
    # Calcular CCF
    ccf_values = ccf(y, x, adjusted=False)[:max_lag+1]
    lags = np.arange(0, max_lag+1)
    
    # Plot
    ax.stem(lags, ccf_values, basefmt=' ')
    ax.axhline(0, color='k', linestyle='-', linewidth=0.8)
    ax.axhline(1.96/np.sqrt(len(y)), color='r', linestyle='--', linewidth=1, 
               label='95% CI')
    ax.axhline(-1.96/np.sqrt(len(y)), color='r', linestyle='--', linewidth=1)
    
    ax.set_xlabel('Lag (meses)', fontsize=11)
    ax.set_ylabel('Correlação', fontsize=11)
    ax.set_title(f'CCF: Ureia vs {translations.get(var, var)}', 
                 fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='best', fontsize=9)
    ax.set_xlim(-0.5, max_lag+0.5)

plt.tight_layout()
# plt.savefig(FIG_DIR / '35_cross_correlation_function.png', dpi=160)
plt.show()

# ============================================================================
# 7. LEAD-LAG PLOTS
# ============================================================================
print("\n" + "="*70)
print("7. LEAD-LAG ANALYSIS")
print("="*70)

lead_lag_vars = ['natural_gas_usd', 'crude_oil_usd']
max_leads = 6

for var in lead_lag_vars:
    if var not in df_merged.columns:
        continue
    
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    axes = axes.flatten()
    
    for lag in range(1, max_leads+1):
        ax = axes[lag-1]
        
        # Criar dados com lag
        df_lag = df_merged[['urea_usd', var]].copy()
        df_lag[f'{var}_lag{lag}'] = df_lag[var].shift(lag)
        df_lag = df_lag.dropna()
        
        x = df_lag[f'{var}_lag{lag}'].values
        y = df_lag['urea_usd'].values
        
        # Scatter
        ax.scatter(x, y, alpha=0.5, s=20, edgecolors='k', linewidths=0.5)
        
        # Linha de tendência
        z = np.polyfit(x, y, 1)
        p = np.poly1d(z)
        x_line = np.linspace(x.min(), x.max(), 100)
        ax.plot(x_line, p(x_line), "r--", linewidth=2)
        
        # Correlação
        corr, pval = pearsonr(x, y)
        
        ax.set_xlabel(f'{translations.get(var, var)} (t-{lag})', fontsize=10)
        ax.set_ylabel('Ureia (t)', fontsize=10)
        ax.set_title(f'Lag {lag}: Corr={corr:.3f}, p={pval:.4f}',
                     fontsize=11, fontweight='bold')
        ax.grid(True, alpha=0.3)
    
    fig.suptitle(f'Lead-Lag Analysis: {translations.get(var, var)} → Ureia',
                 fontsize=14, fontweight='bold', y=1.00)
    plt.tight_layout()
    # plt.savefig(FIG_DIR / f'36_lead_lag_{var}.png', dpi=160)
    plt.show()

# ============================================================================
# 8. TABELA RESUMO DE CORRELAÇÕES
# ============================================================================
print("\n" + "="*70)
print("8. TABELA RESUMO DE CORRELAÇÕES")
print("="*70)

summary_results = []

for var in key_vars:
    if var == 'urea_usd':
        continue
    
    # Correlação em níveis
    data_level = df_merged[['urea_usd', var]].dropna()
    if len(data_level) > 0:
        corr_level, p_level = pearsonr(data_level['urea_usd'], data_level[var])
    else:
        corr_level, p_level = np.nan, np.nan
    
    # Correlação MoM
    data_mom = df_mom[[f'urea_usd_mom', f'{var}_mom']].dropna()
    if len(data_mom) > 0:
        corr_mom_val, p_mom = pearsonr(data_mom[f'urea_usd_mom'], data_mom[f'{var}_mom'])
    else:
        corr_mom_val, p_mom = np.nan, np.nan
    
    # Correlação YoY
    data_yoy = df_yoy[[f'urea_usd_yoy', f'{var}_yoy']].dropna()
    if len(data_yoy) > 0:
        corr_yoy_val, p_yoy = pearsonr(data_yoy[f'urea_usd_yoy'], data_yoy[f'{var}_yoy'])
    else:
        corr_yoy_val, p_yoy = np.nan, np.nan
    
    summary_results.append({
        'variable': var,
        'variable_name': translations.get(var, var),
        'corr_level': corr_level,
        'p_level': p_level,
        'corr_mom': corr_mom_val,
        'p_mom': p_mom,
        'corr_yoy': corr_yoy_val,
        'p_yoy': p_yoy,
        'n_obs': len(data_level)
    })

summary_df = pd.DataFrame(summary_results)
summary_df = summary_df.sort_values('corr_level', ascending=False, key=abs)

# Salvar
summary_df.to_csv(TAB_DIR / '14_correlation_summary.csv', index=False)

print(summary_df.to_string(index=False))
print(f"\n✓ Tabela salva em: {TAB_DIR / '14_correlation_summary.csv'}")

print("\n" + "="*70)
print("ANÁLISE DE CORRELAÇÃO CONCLUÍDA")
print("="*70)
print(f"✓ Gráficos salvos em: {FIG_DIR}")
print(f"✓ Tabelas salvas em: {TAB_DIR}")

In [ ]:
# Gráfico 3: Correlação móvel (24 meses) para entender variação temporal da relação
rc_gas = rolling_corr(s_urea, s_ng, window=24)

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(rc_gas.index, rc_gas.values, linewidth=2, color='C1', label='Correlação móvel (24m)')
ax.axhline(0, color='gray', linestyle='--', linewidth=1, alpha=0.7)
ax.fill_between(rc_gas.index, rc_gas.values, 0, alpha=0.3, color='C1')
ax.set_xlabel('Data')
ax.set_ylabel('Correlação (janela 24m)')
ax.set_title('Correlação Móvel: Ureia vs Gás Natural (24 meses)\nMostra variação temporal da relação', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(loc='best', fontsize=10)

savefig(fig, '22_urea_gas_rolling_correlation.png')

# Estatísticas da correlação móvel
print("\n--- Estatísticas da Correlação Móvel (24m) ---")
print(f"Correlação média: {rc_gas.mean():.4f}")
print(f"Desvio-padrão:    {rc_gas.std():.4f}")
print(f"Mínimo:           {rc_gas.min():.4f} (data: {rc_gas.idxmin()})")
print(f"Máximo:           {rc_gas.max():.4f} (data: {rc_gas.idxmax()})")
print(f"Períodos com corr > 0.5:  {(rc_gas > 0.5).sum()} meses")
print(f"Períodos com corr < -0.5: {(rc_gas < -0.5).sum()} meses")


In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 5.5))

# Eixo esquerdo: Ureia
s_ng = df_merged["gscpi"].astype(float)
ax1.plot(s_urea.index, s_urea.values, color='C0', linewidth=2, label='Ureia (US$/t)')
ax1.set_xlabel('Data')
ax1.set_ylabel('Ureia (US$/t)', color='C0')
ax1.tick_params(axis='y', labelcolor='C0')
ax1.grid(True, alpha=0.3)

ax2 = ax1.twinx()
ax2.plot(s_ng.index, s_ng.values, color='C1', linewidth=2, label='GSCPI')
ax2.set_ylabel('GSCPI', color='C1')
ax2.tick_params(axis='y', labelcolor='C1')

fig.suptitle('Ureia vs GSCPI - Série Temporal (1990-2025)', fontsize=14, fontweight='bold')
# Legenda unificada
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left', fontsize=10)

# savefig(fig, '20_urea_vs_gas_timeseries.png')

plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Config
SEASONAL_PERIOD = 12
series_name = UREA_COL if 'UREA_COL' in globals() else 'urea_usd'

# Target series
s = df_merged[series_name].astype(float).dropna().copy()
s = s[s.index >= pd.to_datetime("1995-01-01")]

# Transforms to test
s_log = np.log(s)
d1 = s.diff().dropna()
d1_log = s_log.diff().dropna()
d12 = s.diff(SEASONAL_PERIOD).dropna()
d12_log = s_log.diff(SEASONAL_PERIOD).dropna()
d1_d12_log = d12_log.diff().dropna()  # both seasonal and first diff

def run_adf(x, autolag="AIC", maxlag=12):
    x = x.dropna()
    stat, p, usedlag, nobs, crit, icbest = adfuller(x.values, autolag=autolag, maxlag=maxlag)
    return {"adf_stat": stat, "adf_p": p, "adf_usedlag": usedlag, "nobs": nobs}

def run_kpss(x, regression="c", nlags="auto"):
    x = x.dropna()
    stat, p, lags, crit = kpss(x.values, regression=regression, nlags=nlags)
    return {"kpss_stat": stat, "kpss_p": p, "kpss_lags": lags, "kpss_reg": regression}

rows = []
tests = {
    "level": s,
    "log(level)": s_log,
    "diff(1)": d1,
    "diff(1, log)": d1_log,
    f"diff({SEASONAL_PERIOD})": d12,
    f"diff({SEASONAL_PERIOD}, log)": d12_log,
    f"diff({SEASONAL_PERIOD}) + diff(1, log)": d1_d12_log,
}

for name, x in tests.items():
    res_adf = run_adf(x)
    res_kpss_c = run_kpss(x, regression="c")   # stationarity around constant (level)
    res_kpss_ct = run_kpss(x, regression="ct") # stationarity around trend
    rows.append({
        "transform": name,
        **res_adf,
        "kpss_c_stat": res_kpss_c["kpss_stat"], "kpss_c_p": res_kpss_c["kpss_p"], "kpss_c_lags": res_kpss_c["kpss_lags"],
        "kpss_ct_stat": res_kpss_ct["kpss_stat"], "kpss_ct_p": res_kpss_ct["kpss_p"], "kpss_ct_lags": res_kpss_ct["kpss_lags"],
        "nobs": res_adf["nobs"],
    })

out = pd.DataFrame(rows).set_index("transform")
print("="*72)
print("Stationarity tests for urea_usd (ADF: H0=unit root | KPSS: H0=stationary)")
print("="*72)
print(out[["adf_stat","adf_p","kpss_c_stat","kpss_c_p","kpss_ct_stat","kpss_ct_p","nobs"]].round(4))

# Save table
out.to_csv(TAB_DIR / "stationarity_urea_tests.csv")
print(f"\n✓ Saved: {TAB_DIR / 'stationarity_urea_tests.csv'}")

# ACF/PACF plots
def acf_pacf_fig(x, title, fname):
    fig, axes = plt.subplots(1, 2, figsize=(11,4))
    plot_acf(x.dropna(), ax=axes[0], lags=36)
    plot_pacf(x.dropna(), ax=axes[1], lags=36, method="ywm")
    axes[0].set_title(f"ACF - {title}")
    axes[1].set_title(f"PACF - {title}")
    fig.tight_layout()
    # fig.savefig(FIG_DIR / fname, dpi=160)
    plt.show()

acf_pacf_fig(s_log, "log(level)", "acf_pacf_urea_log_level.png")
acf_pacf_fig(d1_log, "diff(1, log)", "acf_pacf_urea_log_diff1.png")
acf_pacf_fig(d12_log, f"diff({SEASONAL_PERIOD}, log)", "acf_pacf_urea_log_diff12.png")
acf_pacf_fig(d1_d12_log, f"diff({SEASONAL_PERIOD}) + diff(1, log)", "acf_pacf_urea_log_diff12_diff1.png")

print(f"✓ ACF/PACF saved to: {FIG_DIR}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Configurar estilo para apresentação
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Config
SEASONAL_PERIOD = 12
series_name = UREA_COL if 'UREA_COL' in globals() else 'urea_usd'

# Target series
s = df_merged[series_name].astype(float).dropna().copy()
s = s[s.index >= pd.to_datetime("1995-01-01")]

# Transforms to test
s_log = np.log(s)
d1 = s.diff().dropna()
d1_log = s_log.diff().dropna()
d12 = s.diff(SEASONAL_PERIOD).dropna()
d12_log = s_log.diff(SEASONAL_PERIOD).dropna()
d1_d12_log = d12_log.diff().dropna()

def run_adf(x, autolag="AIC", maxlag=12):
    x = x.dropna()
    stat, p, usedlag, nobs, crit, icbest = adfuller(x.values, autolag=autolag, maxlag=maxlag)
    return {"adf_stat": stat, "adf_p": p, "adf_usedlag": usedlag, "nobs": nobs}

def run_kpss(x, regression="c", nlags="auto"):
    x = x.dropna()
    stat, p, lags, crit = kpss(x.values, regression=regression, nlags=nlags)
    return {"kpss_stat": stat, "kpss_p": p, "kpss_lags": lags}

# ============================================================================
# 1. GRÁFICO: Série Original vs Transformações
# ============================================================================
print("="*70)
print("GRÁFICO 1: Série Original vs Transformações")
print("="*70)

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.patch.set_facecolor('white')

transforms = [
    (s, "Série Original (nível)", axes[0, 0], '#1f77b4'),
    (s_log, "Log da Série", axes[0, 1], '#ff7f0e'),
    (d1_log, "Δ Log (Diff 1º ordem)", axes[1, 0], '#2ca02c'),
    (d12_log, "Δ Log Sazonal (Diff 12 meses)", axes[1, 1], '#d62728'),
]

for series, title, ax, color in transforms:
    ax.plot(series.index, series.values, linewidth=2.5, color=color, alpha=0.85)
    ax.fill_between(series.index, series.values, alpha=0.15, color=color)
    ax.set_title(title, fontsize=14, fontweight='bold', pad=15)
    ax.set_xlabel('Data', fontsize=11)
    ax.set_ylabel('Valor', fontsize=11)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.tick_params(labelsize=10)

fig.suptitle('Transformações da Série Ureia (USD) para Análise de Estacionaridade', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
# savefig(fig, 'stationarity_01_series_transformations.png')
plt.show()

# ============================================================================
# 2. TABELA: Resultados dos Testes Estatísticos
# ============================================================================
print("\n" + "="*70)
print("GRÁFICO 2: Tabela de Testes de Estacionaridade")
print("="*70)

rows = []
tests = {
    "Nível": s,
    "Log(Nível)": s_log,
    "Δ₁ Log": d1_log,
    "Δ₁₂ Log": d12_log,
    "Δ₁ + Δ₁₂ Log": d1_d12_log,
}

for name, x in tests.items():
    res_adf = run_adf(x)
    res_kpss = run_kpss(x, regression="c")
    
    adf_result = "✓ I(0)" if res_adf["adf_p"] < 0.05 else "✗ I(1)"
    kpss_result = "✓ I(0)" if res_kpss["kpss_p"] > 0.05 else "✗ I(1)"
    
    rows.append({
        "Transform": name,
        "ADF p-value": f"{res_adf['adf_p']:.4f}",
        "ADF Result": adf_result,
        "KPSS p-value": f"{res_kpss['kpss_p']:.4f}",
        "KPSS Result": kpss_result,
        "n_obs": res_adf["nobs"],
    })

results_df = pd.DataFrame(rows)

# Criar tabela visual
fig, ax = plt.subplots(figsize=(14, 5))
ax.axis('tight')
ax.axis('off')

table = ax.table(cellText=results_df.values, 
                colLabels=results_df.columns,
                cellLoc='center',
                loc='center',
                colWidths=[0.18, 0.15, 0.15, 0.15, 0.15, 0.12])

table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2.5)

# Colorir header
for i in range(len(results_df.columns)):
    table[(0, i)].set_facecolor('#2c3e50')
    table[(0, i)].set_text_props(weight='bold', color='white')

# Colorir linhas alternadas
colors = ['#ecf0f1', '#ffffff']
for i in range(1, len(results_df) + 1):
    for j in range(len(results_df.columns)):
        table[(i, j)].set_facecolor(colors[i % 2])
        if '✓' in str(table[(i, j)].get_text().get_text()):
            table[(i, j)].set_text_props(color='#27ae60', weight='bold')
        elif '✗' in str(table[(i, j)].get_text().get_text()):
            table[(i, j)].set_text_props(color='#e74c3c', weight='bold')

plt.title('Resumo: Testes de Estacionaridade (ADF vs KPSS)\n' +
          'H₀(ADF) = Raiz Unitária | H₀(KPSS) = Estacionária',
          fontsize=13, fontweight='bold', pad=20)

# savefig(fig, 'stationarity_02_tests_summary.png')
plt.show()

# ============================================================================
# 3. ACF/PACF: Série Original (Não-Estacionária)
# ============================================================================
print("\n" + "="*70)
print("GRÁFICO 3: ACF/PACF - Série Original (LOG)")
print("="*70)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
fig.patch.set_facecolor('white')

plot_acf(s_log.dropna(), ax=axes[0], lags=36, color='#e74c3c', alpha=0.8)
axes[0].set_title('ACF - Log(Série Original)\n→ Decaimento LENTO = NÃO ESTACIONÁRIA', 
                  fontsize=12, fontweight='bold', color='#e74c3c')
axes[0].set_xlabel('Lag (meses)', fontsize=11)
axes[0].set_ylabel('Autocorrelação', fontsize=11)

plot_pacf(s_log.dropna(), ax=axes[1], lags=36, method="ywm", color='#e74c3c', alpha=0.8)
axes[1].set_title('PACF - Log(Série Original)\n→ Picos fortes = Presença de Raiz Unitária', 
                  fontsize=12, fontweight='bold', color='#e74c3c')
axes[1].set_xlabel('Lag (meses)', fontsize=11)
axes[1].set_ylabel('Autocorrelação Parcial', fontsize=11)

fig.suptitle('Diagnóstico: Série Original é NÃO ESTACIONÁRIA', 
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
# savefig(fig, 'stationarity_03_acf_pacf_original.png')
plt.show()

# ============================================================================
# 4. ACF/PACF: Série Diferenciada (Estacionária)
# ============================================================================
print("\n" + "="*70)
print("GRÁFICO 4: ACF/PACF - Série Diferenciada")
print("="*70)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
fig.patch.set_facecolor('white')

plot_acf(d1_log.dropna(), ax=axes[0], lags=36, color='#27ae60', alpha=0.8)
axes[0].set_title('ACF - Δ Log(Série)\n→ Decaimento RÁPIDO = ESTACIONÁRIA', 
                  fontsize=12, fontweight='bold', color='#27ae60')
axes[0].set_xlabel('Lag (meses)', fontsize=11)
axes[0].set_ylabel('Autocorrelação', fontsize=11)

plot_pacf(d1_log.dropna(), ax=axes[1], lags=36, method="ywm", color='#27ae60', alpha=0.8)
axes[1].set_title('PACF - Δ Log(Série)\n→ Corte em lag 1-2 = AR(1) ou AR(2)', 
                  fontsize=12, fontweight='bold', color='#27ae60')
axes[1].set_xlabel('Lag (meses)', fontsize=11)
axes[1].set_ylabel('Autocorrelação Parcial', fontsize=11)

fig.suptitle('Diagnóstico: Série Diferenciada (1ª ordem) é ESTACIONÁRIA', 
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
# savefig(fig, 'stationarity_04_acf_pacf_differenced.png')
plt.show()

# ============================================================================
# 5. ACF/PACF: Série com Diferença Sazonal
# ============================================================================
print("\n" + "="*70)
print("GRÁFICO 5: ACF/PACF - Série com Diferença Sazonal")
print("="*70)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
fig.patch.set_facecolor('white')

plot_acf(d12_log.dropna(), ax=axes[0], lags=36, color='#3498db', alpha=0.8)
axes[0].set_title('ACF - Δ₁₂ Log(Série)\n→ Picos em lag 12, 24 = SAZONALIDADE', 
                  fontsize=12, fontweight='bold', color='#3498db')
axes[0].set_xlabel('Lag (meses)', fontsize=11)
axes[0].set_ylabel('Autocorrelação', fontsize=11)

plot_pacf(d12_log.dropna(), ax=axes[1], lags=36, method="ywm", color='#3498db', alpha=0.8)
axes[1].set_title('PACF - Δ₁₂ Log(Série)\n→ Sazonalidade ainda presente', 
                  fontsize=12, fontweight='bold', color='#3498db')
axes[1].set_xlabel('Lag (meses)', fontsize=11)
axes[1].set_ylabel('Autocorrelação Parcial', fontsize=11)

fig.suptitle('Diagnóstico: Diferença Sazonal Remove Ciclos de 12 Meses', 
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
# savefig(fig, 'stationarity_05_acf_pacf_seasonal.png')
plt.show()

# ============================================================================
# 6. Comparação Visual: Antes vs Depois
# ============================================================================
print("\n" + "="*70)
print("GRÁFICO 6: Comparação - Série Original vs Diferenciada")
print("="*70)

fig, axes = plt.subplots(3, 1, figsize=(16, 10))
fig.patch.set_facecolor('white')

# Original
axes[0].plot(s.index, s.values, linewidth=2.5, color='#e74c3c', alpha=0.8, label='Série Original')
axes[0].fill_between(s.index, s.values, alpha=0.1, color='#e74c3c')
axes[0].set_title('(A) Série Original - Não Estacionária\n(Tendência + Ciclos)', 
                  fontsize=12, fontweight='bold', loc='left')
axes[0].set_ylabel('Preço (US$/t)', fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(labelsize=10)

# Diferenciada 1ª ordem
axes[1].plot(d1_log.index, d1_log.values, linewidth=2, color='#27ae60', alpha=0.8, label='Δ Log')
axes[1].fill_between(d1_log.index, d1_log.values, 0, alpha=0.1, color='#27ae60')
axes[1].axhline(0, color='black', linestyle='-', linewidth=0.8, alpha=0.5)
axes[1].set_title('(B) Δ₁ Log(Série) - Estacionária\n(Remoção da Tendência)', 
                  fontsize=12, fontweight='bold', loc='left')
axes[1].set_ylabel('Log-retorno (%)', fontsize=11)
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(labelsize=10)

# Diferenciada sazonal
axes[2].plot(d1_d12_log.index, d1_d12_log.values, linewidth=2, color='#3498db', alpha=0.8, label='Δ₁ + Δ₁₂ Log')
axes[2].fill_between(d1_d12_log.index, d1_d12_log.values, 0, alpha=0.1, color='#3498db')
axes[2].axhline(0, color='black', linestyle='-', linewidth=0.8, alpha=0.5)
axes[2].set_title('(C) Δ₁ + Δ₁₂ Log(Série) - Máxima Estacionaridade\n(Remoção de Tendência + Sazonalidade)', 
                  fontsize=12, fontweight='bold', loc='left')
axes[2].set_xlabel('Data', fontsize=11)
axes[2].set_ylabel('Log-retorno (%)', fontsize=11)
axes[2].grid(True, alpha=0.3)
axes[2].tick_params(labelsize=10)

fig.suptitle('Transformações Sucessivas para Alcançar Estacionaridade', 
             fontsize=15, fontweight='bold', y=0.995)
plt.tight_layout()
# savefig(fig, 'stationarity_06_transformation_progression.png')
plt.show()

# ============================================================================
# 7. Recomendação para Modelagem
# ============================================================================
print("\n" + "="*70)
print("RECOMENDAÇÃO PARA MODELAGEM")
print("="*70)

recommendation = """
╔════════════════════════════════════════════════════════════════════════════╗
║                    CONCLUSÕES - ESTACIONARIDADE                           ║
╠════════════════════════════════════════════════════════════════════════════╣
║                                                                            ║
║  ✗ SÉRIE ORIGINAL (NÍVEL):                                               ║
║    • Teste ADF: p-value > 0.05 → NÃO REJEITA H₀ (raiz unitária)         ║
║    • ACF com decaimento lento (30+ lags significativos)                  ║
║    • PACF com picos fortes → Não estacionária                            ║
║                                                                            ║
║  ✓ SÉRIE DIFERENCIADA (Δ LOG):                                           ║
║    • Teste ADF: p-value < 0.05 → REJEITA H₀ → Estacionária             ║
║    • ACF decai rapidamente após lag 2-3                                  ║
║    • PACF com corte em lag 1 → Compatível com AR(1)                     ║
║                                                                            ║
║  ⚠ SAZONALIDADE DETECTADA:                                               ║
║    • Picos em lags 12, 24, 36 no ACF/PACF de Δ₁₂ Log                   ║
║    • Recomenda-se Δ₁ + Δ₁₂ Log para máxima estacionaridade              ║
║                                                                            ║
║  📊 MODELO RECOMENDADO:                                                   ║
║    • SARIMA(p,d,q)(P,D,Q)₁₂ com d=1, D=1                                ║
║    • p ∈ {1,2}, q ∈ {0,1}, P ∈ {0,1}, Q ∈ {0,1}                         ║
║    • Validar com Ljung-Box (ACF dos resíduos)                            ║
║                                                                            ║
╚════════════════════════════════════════════════════════════════════════════╝
"""

print(recommendation)

# Salvar recomendações
with open(TAB_DIR / 'stationarity_recommendations.txt', 'w', encoding='utf-8') as f:
    f.write(recommendation)

print(f"\n✓ Recomendações salvas em: {TAB_DIR / 'stationarity_recommendations.txt'}")
print(f"✓ Todos os gráficos salvos em: {FIG_DIR}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

# Carregar dados
df = pd.read_csv(DATA_DIR / 'merged_data_with_trade.csv')
df['date'] = pd.to_datetime(df['date'])

# Exploração inicial dos dados comerciais
print("=" * 80)
print("ANÁLISE DE DADOS COMERCIAIS DISPONÍVEIS")
print("=" * 80)

# Identificar períodos com dados comerciais
trade_cols = ['BRA_tonnes_Export', 'BRA_tonnes_Import', 'BRA_value_usd_Export', 
              'BRA_value_usd_Import', 'CHN_tonnes_Export', 'CHN_tonnes_Import',
              'CHN_value_usd_Export', 'CHN_value_usd_Import']

for col in trade_cols:
    non_null = df[col].notna().sum()
    print(f"{col:25s}: {non_null:4d} observações ({non_null/len(df)*100:5.1f}%)")

print("\nPrimeira data com dados comerciais:", df[df[trade_cols].notna().any(axis=1)]['date'].min())
print("Última data com dados comerciais: ", df[df[trade_cols].notna().any(axis=1)]['date'].max())

# Criar dataset com dados comerciais
df_trade = df[df[trade_cols].notna().any(axis=1)].copy()
df_trade = df_trade.sort_values('date').reset_index(drop=True)

print(f"\nObservações com dados comerciais: {len(df_trade)}")

In [ ]:
# ============================================================================
# GRÁFICO 1: SÉRIE TEMPORAL - PREÇO DA UREIA vs VOLUMES COMERCIAIS (BRASIL)
# ============================================================================

fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# Eixo 1: Preço da Ureia
ax1 = axes[0]
ax1_twin = ax1.twinx()

line1 = ax1.plot(df_trade['date'], df_trade['urea_usd'], 'b-', linewidth=2.5, 
                 label='Preço da Ureia (USD/ton)', alpha=0.8)
ax1.fill_between(df_trade['date'], df_trade['urea_usd'], alpha=0.2, color='blue')
ax1.set_ylabel('Preço da Ureia (USD/ton)', fontsize=11, fontweight='bold', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(True, alpha=0.3)
ax1.set_title('Brasil: Exportações e Importações de Ureia vs Preço (2000-2024)', 
              fontsize=13, fontweight='bold', pad=20)

# Eixo 2: Volumes Brasil
ax2 = axes[1]
ax2.bar(df_trade['date'], df_trade['BRA_tonnes_Export']/1000, alpha=0.7, 
        label='Exportações', color='green', width=20)
ax2.bar(df_trade['date'], -df_trade['BRA_tonnes_Import']/1000, alpha=0.7, 
        label='Importações', color='red', width=20)
ax2.axhline(0, color='black', linestyle='-', linewidth=0.8)
ax2.set_ylabel('Volumes (mil toneladas)', fontsize=11, fontweight='bold')
ax2.legend(loc='upper left', fontsize=10)
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_title('Brasil: Volumes de Comércio de Ureia', fontsize=12, fontweight='bold')

# Eixo 3: Valores Brasil
ax3 = axes[2]
ax3.bar(df_trade['date'], df_trade['BRA_value_usd_Export']/1e6, alpha=0.7, 
        label='Valor Exportações', color='darkgreen', width=20)
ax3.bar(df_trade['date'], -df_trade['BRA_value_usd_Import']/1e6, alpha=0.7, 
        label='Valor Importações', color='darkred', width=20)
ax3.axhline(0, color='black', linestyle='-', linewidth=0.8)
ax3.set_ylabel('Valor (milhões USD)', fontsize=11, fontweight='bold')
ax3.set_xlabel('Data', fontsize=11, fontweight='bold')
ax3.legend(loc='upper left', fontsize=10)
ax3.grid(True, alpha=0.3, axis='y')
ax3.set_title('Brasil: Valores Comerciais de Ureia', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('01_brasil_comercio_ureia.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Gráfico 1 salvo: 01_brasil_comercio_ureia.png")

In [ ]:
# ============================================================================
# GRÁFICO 2: SÉRIE TEMPORAL - PREÇO DA UREIA vs VOLUMES COMERCIAIS (CHINA)
# ============================================================================

fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# Eixo 1: Preço da Ureia
ax1 = axes[0]
line1 = ax1.plot(df_trade['date'], df_trade['urea_usd'], 'b-', linewidth=2.5, 
                 label='Preço da Ureia (USD/ton)', alpha=0.8)
ax1.fill_between(df_trade['date'], df_trade['urea_usd'], alpha=0.2, color='blue')
ax1.set_ylabel('Preço da Ureia (USD/ton)', fontsize=11, fontweight='bold', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.grid(True, alpha=0.3)
ax1.set_title('China: Exportações e Importações de Ureia vs Preço (2000-2024)', 
              fontsize=13, fontweight='bold', pad=20)

# Eixo 2: Volumes China
ax2 = axes[1]
ax2.bar(df_trade['date'], df_trade['CHN_tonnes_Export']/1000, alpha=0.7, 
        label='Exportações', color='#FF6B6B', width=20)
ax2.bar(df_trade['date'], -df_trade['CHN_tonnes_Import']/1000, alpha=0.7, 
        label='Importações', color='#4ECDC4', width=20)
ax2.axhline(0, color='black', linestyle='-', linewidth=0.8)
ax2.set_ylabel('Volumes (mil toneladas)', fontsize=11, fontweight='bold')
ax2.legend(loc='upper left', fontsize=10)
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_title('China: Volumes de Comércio de Ureia', fontsize=12, fontweight='bold')

# Eixo 3: Valores China
ax3 = axes[2]
ax3.bar(df_trade['date'], df_trade['CHN_value_usd_Export']/1e6, alpha=0.7, 
        label='Valor Exportações', color='#C92A2A', width=20)
ax3.bar(df_trade['date'], -df_trade['CHN_value_usd_Import']/1e6, alpha=0.7, 
        label='Valor Importações', color='#1971C2', width=20)
ax3.axhline(0, color='black', linestyle='-', linewidth=0.8)
ax3.set_ylabel('Valor (milhões USD)', fontsize=11, fontweight='bold')
ax3.set_xlabel('Data', fontsize=11, fontweight='bold')
ax3.legend(loc='upper left', fontsize=10)
ax3.grid(True, alpha=0.3, axis='y')
ax3.set_title('China: Valores Comerciais de Ureia', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('02_china_comercio_ureia.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Gráfico 2 salvo: 02_china_comercio_ureia.png")